# Создание базы данных

In [15]:
import requests
from bs4 import BeautifulSoup
import csv

# Создаем файл CSV и записываем заголовки столбцов
with open('hse_data_2022.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Название ВКР", "Студент", "Полученное образование", "Год"])

# Проходим по всем страницам
for page in range(1, 118):
    url = f"https://scholar.google.ru/scholar?hl=en&as_sdt={page}&q=%D0%9F%D0%BE%D0%BB%D0%B8%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F+%5BPDF%5D&scisbd=1"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # На каждой странице находим все выпускные работы
    works = soup.find_all('li', class_='vkr-card vkr-list__item')

    # Из каждой работы извлекаем необходимую информацию
    for work in works:
        title = work.find('a', class_='link').text.strip()
        student = work.find_all('span', class_='vkr-card__value')[1].text.strip()
        program = work.find('span', class_='vkr-card__value').text.strip()
        year = work.find_all('span', class_='vkr-card__value')[-1].text.strip()

        # Записываем информацию в файл CSV
        with open('hse_data_2022.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([title, student, program, year])
            
# Проделываем то же самое для ВКР 2023 года
with open('hse_data_2023.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Название ВКР", "Студент", "Полученное образование", "Год"])

for page in range(1, 118): 
    url = f"https://www.hse.ru/edu/vkr/?faculty=122842106&year=2023&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    works = soup.find_all('li', class_='vkr-card vkr-list__item')

    for work in works:
        title = work.find('a', class_='link').text.strip()
        program = work.find('span', class_='vkr-card__value').text.strip()
        student = work.find_all('span', class_='vkr-card__value')[1].text.strip()
        year = work.find_all('span', class_='vkr-card__value')[-1].text.strip()

        with open('hse_data_2023.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([title, student, program, year])

In [22]:
df1 = pd.read_csv('hse_data_2022.csv')
df2 = pd.read_csv('hse_data_2023.csv')

combined_df = pd.concat([df1, df2])
combined_df = combined_df.drop(['Студент', 'Полученное образование', 'Год'], axis=1)

combined_df.to_csv('hse_data_2022_2023_nlp.csv', index=False)

# Применение YandexGPT

In [1]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm

def send_to_yandex_api(request_text):
    # URL для Yandex API
    url = 'https://llm.api.cloud.yandex.net/llm/v1alpha/instruct'
    # API ключ (должен быть скрыт/защищён в реальных условиях)
    api_key = "Api-Key AQVNyVqBi-XoJ1cAo7VIxq6ztgXm3owqowtso5Qb"
    # Заголовки запроса
    headers = {
        "Authorization": api_key,
    }
    # Тело запроса в формате JSON
    payload = {
        "model": "general",
        "instruction_text": "Проанализируй следующие названия ВКР и выдели ключевые темы, указав, как часто они встречаются. Представь результаты в удобочитаемом формате.",
        "request_text": f"{request_text}",
        "generation_options": {
            "max_tokens": 10000,  
            "temperature": 0.4
        }
    }
    retries = 0
    max_retries=3
    while retries < max_retries:
        try:
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 200:
                return response.json()
            else:
                return {'error': response.status_code, 'message': response.text}
        except requests.exceptions.RequestException as e:
            retries += 1
            if retries < max_retries:
                print(f"Request failed, retrying... (Attempt {retries}/{max_retries})")
                time.sleep(5)
            else:
                return {'error': 'RequestException', 'message': str(e)}

# Чтение CSV файла с запросами
df = pd.read_csv('hse_data_2022_2023_nlp.csv')

# Объединяем все названия ВКР в один большой текст для анализа
all_titles = " ".join(df['Название ВКР'].tolist())

response = send_to_yandex_api(all_titles)
if 'result' in response:
    analyzed_topics = response['result']  # Получение анализа тем
else:
    analyzed_topics = response  # Сохранение ошибки, если таковая произошла

# Сохранение результатов в JSON файл
with open('analyzed_themes.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps({"analyzed_topics": analyzed_topics}, ensure_ascii=False))

In [2]:
with open('analyzed_themes.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
print(data)

{'Тема 0 (Государство и Управление)': 'разработка россии особенности программы роль выгорания компании сотрудников государственных государственного', 'Тема 1 (Общественное здоровье и Социальная политика)': 'россии анализ влияние идентичности социально 19 формирование факторы сравнительный covid', 'Тема 2 (Социальное поведение)': 'российской федерации роль россии примере людей поведения анализ фактор социальные', 'Тема 3 (Идентичность и Организационное развитие)': 'особенности анализ идентичности жизни влияние организации развития факторы восприятие россии', 'Тема 4 (Психоанализ и Поведенческие исследования)': 'примере аспекты психоаналитические исследование психоаналитическое роль организации психоаналитический влияния работе', 'Тема 5 (Психология и Организационное поведение)': 'психоаналитический взгляд примере факторы исследование влияние сотрудников психологические организации личности', 'Тема 6 (Гендер и Личностное развитие)': 'примере роль личности исследование женщин разработка а

# Преобработка данных

In [38]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Чтение данных
df = combined_df

# Получение стандартного списка стоп-слов на русском языке и добавление новых слов
russian_stopwords = stopwords.words('russian')
extra_stopwords = ['россия', 'россии', 'российский', 'российской', 'федерации', 'российских', 'анализ']  # Список добавляемых стоп-слов
russian_stopwords.extend(extra_stopwords)

def preprocess_text(text):
    text = text.lower()  # приведение к нижнему регистру
    text = re.sub(r'\d+', '', text)  # удаление цифр
    text = re.sub(r'[«»]', '', text)  # удаление « и »
    text = text.translate(str.maketrans('', '', string.punctuation))  # удаление пунктуации
    tokens = word_tokenize(text)  # токенизация
    tokens = [word for word in tokens if not word in russian_stopwords]  # удаление стоп-слов
    return tokens

# Применение предобработки к каждому тексту
df['processed_text'] = df['Название ВКР'].apply(preprocess_text)


# Подготовка данных для LDA и LSI

In [39]:
from gensim import corpora, models

# Подготовка данных для LSI и LDA
dictionary = corpora.Dictionary(df['processed_text'])
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]

# LDA и LSI

In [40]:
# LSI Model
lsi_model = models.LsiModel(corpus, id2word=dictionary, num_topics=10)
lsi_topics = lsi_model.print_topics(num_words=10)

# LDA Model
lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=10)
lda_topics = lda_model.print_topics(num_words=10)

In [41]:
# Функция для красивого вывода тем LSI и LDA
def print_topics(model, model_name):
    print(f"Темы из модели {model_name}:")
    for i, topic in enumerate(model.print_topics(num_words=5)):
        words = ', '.join(word for word, prob in model.show_topic(i))
        print(f"Тема {i + 1}: {words}")

# Вывод тем из LSI
print_topics(lsi_model, "LSI")

Темы из модели LSI:
Тема 1: примере, психоаналитический, взгляд, роль, влияние, личности, особенности, исследование, аспекты, организации
Тема 2: психоаналитический, взгляд, примере, подход, роль, развитие, влияние, аспекты, особенности, личности
Тема 3: влияние, примере, особенности, роль, факторы, развития, сотрудников, государственной, разработка, психоаналитический
Тема 4: влияние, особенности, роль, разработка, факторы, сотрудников, примере, развития, организации, программы
Тема 5: роль, особенности, личности, сотрудников, разработка, факторы, психоаналитический, развития, развитии, организации
Тема 6: исследование, психоаналитическое, личности, примере, влияния, аспекты, лидера, работе, психоаналитические, влияние
Тема 7: особенности, разработка, сотрудников, факторы, программы, тренинга, развития, организации, развитию, компании
Тема 8: аспекты, психоаналитические, исследование, организации, работе, группах, психоаналитического, психоаналитическое, примере, динамики
Тема 9: факт

In [42]:
# Вывод тем из LDA
print_topics(lda_model, "LDA")

Темы из модели LDA:
Тема 1: особенности, ценностей, разработка, примере, организации, уровне, коммуникации, проблемы, политической, внедрения
Тема 2: примере, гг, взгляд, психоаналитический, восприятие, медицинских, пандемии, стратегии, сфере, региональной
Тема 3: исследование, кризис, роль, социальных, современной, примере, подход, образа, гане, подростков
Тема 4: психоаналитический, примере, подход, взгляд, влияние, коучингового, взаимосвязь, профессиональной, фильма, деятельности
Тема 5: примере, практики, оценки, аспекты, психоаналитические, развития, специалистов, фактор, молодых, особенности
Тема 6: особенности, психоаналитические, лидера, влияния, развития, фактор, государственной, аспекты, команду, связь
Тема 7: факторы, примере, социальные, психологическое, представления, сотрудников, влияние, кризиса, сети, фактор
Тема 8: влияние, особенности, отца, примере, феноменология, психоаналитически, коучинге, ориентированном, отсутствующего, фигура
Тема 9: психоаналитический, взгляд,

# ruBERT 

In [44]:
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

def bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Функция для преобразования всех текстов в BERT векторы
def texts_to_bert_vectors(texts):
    vectors = []
    for text in texts:
        vector = bert_embedding(' '.join(text))
        vectors.append(vector[0])
    return np.array(vectors)

# Преобразование всех текстов в датасете
bert_vectors = texts_to_bert_vectors(df['processed_text'])

# Использование KMeans для кластеризации векторов
kmeans = KMeans(n_clusters=10)  # предположим, что мы хотим извлечь 10 тем
kmeans.fit(bert_vectors)

# Получение и вывод основных тем
for i in range(10):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    cluster_texts = df['processed_text'].iloc[cluster_indices]
    print(f"Тема {i + 1}:")
    for text in cluster_texts.sample(min(len(cluster_texts), 5)):  # Вывести несколько примеров из каждого кластера
        print(f" - {text[:100]}...")  # Выводим начало текста для представления темы
    print("\n")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum lengt

Тема 1:
 - ['нормативные', 'представления', 'европейских', 'популистов', 'народе', 'дискурсе', 'региональных', 'национальных', 'наднациональных', 'предвыборных', 'кампаний']...
 - ['влияние', 'традиционной', 'идеологии', 'маскулиности', 'следование', 'правилам']...
 - ['политический', 'режим', 'фактор', 'эффективности', 'реакции', 'правительств', 'пандемию', 'covid']...
 - ['жилищный', 'портрет', 'москвы', 'динамика', 'жилой', 'застройки', 'её', 'влияние', 'образ', 'городских', 'районов']...
 - ['патронклиентские', 'отношения', 'силовых', 'элитах', 'постсоветских', 'трансформаций']...


Тема 2:
 - ['психоаналитическая', 'семантика', 'смыслы', 'символы', 'структуры']...
 - ['нарциссизма', 'смерти', 'нарциссизму', 'жизни', 'примере', 'индивидуальных', 'случаев', 'психоаналитического', 'консультирования', 'коучинга']...
 - ['психоаналитическое', 'исследование', 'личности', 'ницше', 'влияния', 'идей', 'формирование', 'общественного', 'сознания']...
 - ['молчание', 'деперсонализация', 'псих